# Transformer

## Self-Attention

1. 入力要素$x^{(i)}$と集合$\{1, ..., T\}$の$j$番目の要素ごとにドット積$\{ {U_q x}^{(i)} \} ^T {U_k}x^{(j)}$を計算する。
2. ソフトマックス関数を使ってこれらのドット積を正規化することで、Attention重み$\alpha_{ij}$を求める。
3. 入力シーケンス全体の加重和として出力$z^{(i)}$を計算する。

`a.matmul(b)` は、数式でいうと **「左から右」** に掛けており、つまり$a\cdot b$となる。

`torch.matmul(a, b)` は、**入力の次元数（テンソルのrank）に応じて異なる方法で計算**が行われる、**多機能な行列／テンソル積関数**。

| 入力形状 `a`  | 入力形状 `b`  | 結果形状      | 説明        |
| --------- | --------- | --------- | --------- |
| `(n,)`    | `(n,)`    | `()`      | ベクトルの内積   |
| `(m,n)`   | `(n,p)`   | `(m,p)`   | 行列積       |
| `(n,)`    | `(n,p)`   | `(p,)`    | ベクトル × 行列 |
| `(m,n)`   | `(n,)`    | `(m,)`    | 行列 × ベクトル |
| `(b,m,n)` | `(b,n,p)` | `(b,m,p)` | バッチ行列積    |


| 関数             | 入力次元       | 対応                | 自動ブロードキャスト  |
| -------------- | ---------- | ----------------- | ----------- |
| `torch.matmul` | 任意（1D～多次元） | 行列・ベクトル・バッチ積など全対応 | あり          |
| `torch.bmm`    | 必ず3D       | 3Dテンソル（バッチ行列積）のみ  | なし（サイズ一致必須） |


## デコーダー
### Masked Multi-Head Attention

未出現の部分にはマスクをして学習する。  
出力埋め込み->Masked Multi-Head Attention->Multi-Head Attention（エンコードされた入力が入る）->全結合->  

### 位置エンコーディング
* 絶対エンコーディング
* 相対エンコーディング

### 層正規化
ミニバッチのサイズの制約や依存性を解消する。

---

## 大規模言語モデル

Transformerベースのモデルの訓練の処理全体は次の2つの部分（ステージ）で構成される。
1. ラベルなしの大規模なデータセットを使ってモデルの事前訓練を行う。(教師なし事前学習（自己教師あり学習）)
2. ラベル付きのデータセットを使って具体的なダウンストリーム（下流工程の）タスクのためにモデルの訓練を行う（つまり、ファインチューニングを行う）。

| モデル      | 主な活用法      | 学習の流れ                              | 下流との関係                |
| -------- | ---------- | ---------------------------------- | --------------------- |
| **ELMo** | 特徴量抽出      | BiLSTMで事前学習 → 出力ベクトルを固定して使用        | 明示的に**特徴量**として「下流」に渡す |
| **BERT** | ファインチューニング | Transformerで事前学習 → タスクに応じて全モデルを再学習 | モデル全体が**下流タスクと統合**される |


#### GPTモデル

| モデル   | リリース | パラメーターの数     | 論文タイトル                                                                 |
|----------|----------|-----------------------|------------------------------------------------------------------------------|
| GPT-1    | 2018年   | 110,000,000（1.1億）   | [Improving Language Understanding by Generative Pre-Training](https://www.mikecaptain.com/resources/pdf/GPT-1.pdf)            |
| GPT-2    | 2019年   | 1,500,000,000（15億）  | [Language Models are Unsupervised Multitask Learners](https://storage.prod.researchhub.com/uploads/papers/2020/06/01/language-models.pdf)             |
| GPT-3    | 2020年   | 175,000,000,000（1750億）| [Language Models are Few-Shot Learners](https://proceedings.neurips.cc/paper_files/paper/2020/file/1457c0d6bfcb4967418bfb8ac142f64a-Paper.pdf)                                  |


---

#### GPT-1
デコーダー（エンコーダーブロックなし）と追加の層で構成されたTrandformerとして考えることができる。この追加の層は、特定のタスクを実行するための教師ありファインチューニングで追加される。

**GPT-1の出力は単語予測だけなのに、なぜ追加層だけで様々なタスクに対応できるのか?**

1. **言語モデルの出力は柔軟な形式を学習している**

* 次の単語を予測するというタスクは、**非常に一般的なフォーマット**です。
* 例えば、文章分類・翻訳・要約などもすべて、「ある入力に対して適切な出力文を生成する」タスクに変換できます。  
例：入力: "The movie was great. Sentiment:", 出力: "Positive"

1. **事前学習で得られた汎用的な文脈理解**

* 「次の単語予測」の繰り返しによって、文脈の理解や単語間の関係性を自然に学習しています。
* これにより、事前学習済みモデルには既に**言語的知識・構文・語彙意味**などが詰め込まれており、**ちょっとした調整で別のタスクに対応できる素地がある**のです。

3. **追加層によるタスク特化**

* 例えば分類タスクでは、Transformerの最終出力（例えば文末のベクトルなど）を使って**Softmaxでラベルを予測**するような層を追加します。
* この追加層（fine-tuning head）だけをタスクに応じて入れ替えることで、**多くの自然言語処理タスクに対応可能**になります。

| 項目     | 内容                                    |
| ------ | ------------------------------------- |
| モデル本体  | 一貫して単語予測（次のトークン予測）                    |
| 汎用性の理由 | 文脈理解能力が豊かで、追加層だけで多様な出力に対応可能           |
| タスク適応  | 追加の教師ありfine-tuning層で、出力形式をタスクに合わせて変える |

---

#### GPT-2
GPTの適応力を見て、これはいけると思った研究者がタスク特有の入力とモデル設定を取り除いたことがGPT-2の開発に繋がった。GPT-1とは異なり、GPT-2は入力でもファインチューニングでも追加の変更を一切要求しない。必要なフォーマットに合わせて文を整理し直さなくても、GPT-2はさまざまな種類の入力を見分け、ほんのわずかなヒント（いわゆるコンテキスト）だけを該当するダウンストリームタスクを実行できる。このようなことが可能なのは、モデルの出力確率が（入力だけではなく）入力とタスクの種類の両方を条件とする条件付き確率$p(output|input, task)$だからである。  
すごい！

#### GPT-3
few-shot学習にシフト。Sparse Attentionなどにより効率化。

---

## GPT-2を使って新しいテキストを生成

In [1]:
from transformers import pipeline, set_seed
# gpt2モデルのファイルをHugging Faceのサーバーから自動的にダウンロードしている（約500 MB？)。
generator = pipeline('text-generation', model='gpt2')

/Users/nagairyousuke/名称未設定フォルダ/StudyMLList/StudyML/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


In [2]:
set_seed(123)
generator("Hey readers, today is", max_length=20, num_return_sequences=3, truncation=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


KeyboardInterrupt: 

In [ ]:
# 入力テキストから特徴量を生成する方法
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
text = "Let us encode this sentence"
encoded_input = tokenizer(text, return_tensors='pt')
encoded_input

{'input_ids': tensor([[ 5756,   514, 37773,   428,  6827]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [ ]:
from transformers import GPT2Model
model = GPT2Model.from_pretrained('gpt2')
outputs = model(**encoded_input)

In [ ]:
outputs['last_hidden_state'].shape  # (batch_size, sequence_length, hidden_size)

torch.Size([1, 5, 768])

## BERT(Bidrectional Encoder Representations Transformers)
パラメータ：345,000,000(GPT-2の1/5)  
特定の単語のエンコーディングがその前にある単語と後ろにある単語の両方によって決まる。双方向の訓練により、文を単語ごとに生成するという能力は奪われるものの、BERTモデルは情報を双方向に処理できるため、分類といった他のタスクで高品質な入力エンコーディングを提供する。

### 訓練
BERTエンコーダー：トークン埋め込み+セグメント埋め込み+位置埋め込み  
事前訓練には**MLM**(masked language modeling)と**次文予測**の2つの教師なし学習タスクが含まれる。
* 次文予測  
BERTが2つの文が論理的に連続しているかどうかを判定するタスク

* MLM  
入力文のうち15%のトークンをランダムに対象とし、以下のように加工：

| 処理  | 割合            | 例                              |
| --- | ------------- | ------------------------------ |
| 80% | `[MASK]`に置き換え | "I love \[MASK] learning."     |
| 10% | ランダムな単語に置き換え  | "I love **banana** learning."  |
| 10% | 元のまま残す        | "I love **machine** learning." |
2. 「[MASK]があれば必ずそれが予測対象とは限らない」
3. 「たとえ[MASK]があっても、どの単語が重要か文脈を全体で判断しなければならない」
---

| 観点     | 内容                            |
| ------ | ----------------------------- |
| 最終的な目標 | 汎用的な**言語理解モデル**を構築すること        |
| 学習するもの | 文脈を理解する**トークンや文のベクトル表現**      |
| 使い道    | 文分類、QA、NER、翻訳、要約、感情分析など多岐にわたる |
| 革新的な点  | 学習済みモデルを「**再利用できる**」という大きな汎用性 |

```Bash
入力（例: "I love NLP."）：
[CLS] I love NLP . [SEP]
 ↓    ↓ ↓    ↓  ↓   ↓
[101, 146, 1567, 2034, 119, 102]（← input_ids）
  ↓   ↓   ↓   ↓   ↓
BERTのEncoderが処理
  ↓   ↓   ↓   ↓   ↓
出力ベクトル（各トークンごとに768次元）
 → 線形層 → Softmax → [確率1, 確率2, ..., 確率30000]
[CLS]トークンの出力（=文全体のベクトル）もここに含まれる


## BART(Bidirectional and Auto-Regressive Transformer)
GPTがTransformerのデコーダ構造を活用するのに対し、BERTがTransformerのエンコーダ構造を活用する。よって、GPTがテキストの生成を得意とするのに対し、BERTは分類タスクに適している。BERTは「穴埋め」で、GPTは「事後予測」である。BARTについては、GPTとBERTの両方を一般化したものとして考えることができる。  
BARTは双方向エンコーダと自己回帰デコーダでできている。  
勘違いしてたけど、GPTやBERTはそれぞれ用途が異なるため、一概に優劣はつけにくい。

| モデル名     | アーキテクチャ            | 目的 | 方向性              | 主な用途            |
| -------- | ------------------ | -- | ---------------- | --------------- |
| **BERT** | Encoder-only（双方向）  | 理解 | 文全体を見て\[MASK]を予測 | 分類、QA、NERなど     |
| **GPT**  | Decoder-only（一方向）  | 生成 | 左→右の単語生成         | 会話、生成、要約、翻訳など   |
| **PaLM** | Decoder-only（GPT型） | 生成 | 自己回帰型（GPTと同じ）    | 文章生成、推論、少数例学習など |

## PyTorchでのBERTモデルのファインチューニング

BERTはモデルの知名度が非常に高く、それでいてGPUが1つだけでもファインチューニングを実行できる扱いやすいサイズである点でバランスが良い。  
DistilBERT：事前訓練済みのBERTモデルをベースモデルとし、このモデルを抽出することで作成された軽量なTransformerモデル。元のモデルには、110,000,000個あまりのパラメータが含まれているが、40%ほど少なくなっている。また、60%高速である一方、GLUEでBERTの95%の性能を維持している。

### IMDb映画レビューデータセット

In [ ]:
import gzip
import shutil
import time

import pandas as pd
import requests
import torch
import torch.nn.functional as F
import torchtext

import transformers
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForTokenClassification

In [ ]:
torch.backends.cudnn.deterministic = True
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    DEVICE = torch.device("mps")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
NUM_EPOCHS = 3

In [ ]:
path='../第8章_機械学習の適用_感情分析'
df = pd.read_csv(f'{path}/movie_data.csv')
df.head()